In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

pd.pandas.set_option('display.max_columns', None)

import warnings
warnings.simplefilter(action='ignore')

In [2]:
lending_train = pd.read_csv('lending_train.csv')
lending_val = pd.read_csv('lending_val.csv')

print(lending_train.shape)
lending_train.head()

(381727, 100)


,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,mths_since_last_delinq_na,mths_since_last_record_na,mths_since_last_major_derog_na,il_util_na,mths_since_recent_bc_dlq_na,mths_since_recent_inq_na,mths_since_recent_revol_delinq_na,num_tl_120dpd_2m_na,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,home_ownership_Rare,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_Rare,pymnt_plan_n,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Rare,disbursement_method_Cash,disbursement_method_Rare,debt_settlement_flag_N,debt_settlement_flag_Rare
0,15000.0,15000.0,14975.0,10.91,490.45,95000.0,5.70,1.0,2.0,3.0,91.0,12.0,1.0,5510.0,20.4,20.0,0.0,44.0,0.0,0.0,181778.0,1.0,1.0,0.0,0.0,31.0,15878.0,87.0,1.0,5.0,2119.0,47.0,27000.0,3.0,2.0,3.0,7.0,15148.0,20524.0,14.8,0.0,0.0,147.0,120.0,1.0,1.0,2.0,1.0,3.0,5.0,3.0,0.0,4.0,6.0,8.0,8.0,5.0,10.0,13.0,6.0,12.0,0.0,0.0,0.0,1.0,95.0,12.5,1.0,0.0,209256.0,21388.0,24100.0,18281.0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0
1,1000.0,1000.0,1000.0,15.05,34.69,78000.0,10.51,0.0,2.0,31.0,76.0,16.0,0.0,25277.0,45.1,25.0,0.0,44.0,0.0,0.0,240899.0,1.0,1.0,1.0,1.0,5.0,2720.0,91.0,0.0,2.0,6654.0,48.0,56000.0,2.0,0.0,3.0,3.0,17207.0,24623.0,50.7,0.0,0.0,56.0,212.0,20.0,5.0,2.0,20.0,35.0,5.0,32.0,0.0,6.0,6.0,10.0,13.0,3.0,14.0,20.0,6.0,16.0,0.0,0.0,0.0,1.0,100.0,60.0,0.0,0.0,283000.0,27997.0,49900.0,3000.0,1,1,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0
2,12000.0,12000.0,12000.0,15.05,285.80,103600.0,27.49,1.0,2.0,15.0,76.0,19.0,0.0,17322.0,35.4,31.0,0.0,44.0,0.0,1878.0,213471.0,3.0,4.0,2.0,6.0,3.0,51002.0,72.0,1.0,4.0,6414.0,57.0,49000.0,2.0,2.0,3.0,10.0,11860.0,17666.0,45.0,0.0,0.0,166.0,86.0,5.0,3.0,2.0,19.0,35.0,1.0,15.0,0.0,5.0,9.0,6.0,6.0,15.0,14.0,14.0,9.0,19.0,0.0,0.0,0.0,3.0,96.7,16.7,0.0,0.0,280465.0,68324.0,32100.0,70647.0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,1,1,0,1,0,1,0,1,0
3,5000.0,5000.0,5000.0,12.62,167.56,43000.0,10.66,0.0,0.0,31.0,88.0,7.0,1.0,2644.0,57.5,15.0,0.0,44.0,0.0,492.0,5304.0,1.0,1.0,0.0,1.0,18.0,2660.0,57.0,1.0,3.0,2048.0,57.0,4600.0,0.0,0.0,0.0,4.0,884.0,186.0,92.8,0.0,0.0,141.0,212.0,5.0,5.0,0.0,24.0,35.0,23.0,32.0,0.0,2.0,3.0,2.0,4.0,5.0,6.0,10.0,3.0,7.0,0.0,0.0,0.0,1.0,100.0,50.0,1.0,0.0,9275.0,5304.0,2600.0,4675.0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,0,1,0
4,8000.0,8000.0,8000.0,10.49,259.99,62000.0,9.19,0.0,1.0,41.0,76.0,11.0,0.0,3560.0,31.0,13.0,0.0,41.0,0.0,0.0,40704.0,2.0,8.0,2.0,2.0,5.0,37144.0,96.0,1.0,2.0,1848.0,81.0,11500.0,4.0,1.0,4.0,4.0,3700.0,7940.0,31.0,0.0,0.0,81.0,167.0,5.0,5.0,1.0,5.0,35.0,5.0,41.0,2.0,3.0,3.0,3.0,3.0,8.0,3.0,4.0,3.0,11.0,0.0,0.0,0.0,3.0,84.6,33.3,0.0,0.0,50272.0,40704.0,11500.0,38772.0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,0,1,0


In [3]:
rate_train = lending_train['int_rate']
rate_val = lending_val['int_rate']

lending_train.drop('int_rate', axis=1, inplace=True)
lending_val.drop('int_rate', axis=1, inplace=True)

## Feature Selection
### Lasso

We will do a rough feature selection using a Lasso linear model.

In [5]:
sel_ = SelectFromModel(Lasso(alpha=1.5, random_state=3))
sel_.fit(lending_train, rate_train)

sel_feats = lending_train.columns[(sel_.get_support())]
print(f'Number of selected features: {len(sel_feats)}')
print(sel_feats)

Number of selected features: 26
Index(['loan_amnt', 'funded_amnt_inv', 'installment', 'dti',
       'mths_since_last_delinq', 'revol_bal', 'revol_util', 'tot_coll_amt',
       'mths_since_rcnt_il', 'il_util', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal',
       'bc_open_to_buy', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mths_since_recent_bc', 'mths_since_recent_inq', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'total_il_high_credit_limit'],
      dtype='object')


In [6]:
# save selected features
pd.Series(sel_feats).to_csv('lasso_sel_feats.csv', index=False)